# Ноутбук с экспериментами

# Вывод

Сразу кратко опишу все проведенные мной эксперименты, чтобы не приходилось искать выводы в конце или далее по тексту.

Сначала, я создал один большой вектор из всех числовых признаков, созданных мной ранее различный статистик текстовых данных, которые описаны ниже. Также я возлагал надежды на векторы полученные из частотных словарей см. в описаниях признаков.<br>
Я предсказывал классы через косинусную близость полученных векторов.<br>
Затем попробовал уменьшить размерности PCA.<br>
Затем я стал пробовать на этих векторах линейные модели в пайплайнах с различными скалерами и гиперпараметрами.<br>
Затем я попробовал обучать на BoW и Tfidf.<br>
С Tfidf модели стали показывать лучшие результаты и далее я стал экспериментировать уже с этими данными, пробовал объединять с полученным в начале вектором, но не получил прироста. <br>
Лучшую модель я получил в итоге с пайплайном TfidfVectorizer(ngram_range=(1, 2)) с OneVsRestClassifier, MaxAbsScaler и LogisticRegression.<br>
Далее я пробовал этот же пайплайн но с ngram_range=(1, 1) и MultinomialNB, но получил результат похуже.<br>
Далее я пробовал стекинг лучших моделей LR и NB c BoW и Tfidf, результат не улучшился. StackingClassifier не работал на классах с одним объектом, пришлось его перенастраивать. <br>
Я пробовал сложить вероятности классов лучших моделей и усреднить, но это не помогло.  <br>
Затем, перейдя на Kaggle я попробовал обучать на векторах SentenceTransformer, TinyBert и Word2Vec, модели LR и Catboost с использованием GPU, на этих признаках по отдельности, вместе и с различными комбинациями (с включениями TFidf и вектора на статистиках) но эти модели не показали более высокий результат. <br>
Пробовал отобрать наиболее значимые признаки при помощи RFECV, но не вышло, из-за недостаточной памяти. <br>
Возможно, что результат удастся улучшить, если попробовать дообучить лучшую модель на каких-то векторах из этих экспериментов. Или использовать ансамбль из Catboost с векторами и лучшей модели.

In [ ]:
import re
import string
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ссылки на файлы с данными:<br>
[df_train.pq](https://drive.google.com/file/d/1-8MfMUBhrec_Nqsh_4NFYhtoeY3hZg1z/view?usp=sharing)<br>
[df_test.pq](https://drive.google.com/file/d/1vMGRqYBuR8PD6owZg1UACS4Ou_biPTZt/view?usp=sharing)

In [ ]:
df_train = pd.read_parquet('//content//drive//MyDrive//AI//NLP//df_train.pq')
df_test = pd.read_parquet('//content//drive//MyDrive//AI//NLP//df_test.pq')

## Описание признаков тренировочного и тестового датасетов
1.  'author' - имя и фамилия автора **(целевая переменная 100 классов)**
2.  'text_' - оригинальный полный текст книги без пробельных символов и метаданных
3.  'cnt_sent' - количество предложений
4.  'text_len' - количество символов в оригинальном тексте, до обрезки
5.  'text' - обрезанный в середине текст книги до максимальной длинны в 200000 слов
6.  'text_len2' - количество символов в "text"
7.  'words_cnt' - количество слов в обрезанном тексте
8.  'wrds_sent_cnt' - отношение количества слов к количеству предложений, или средняя длина предложения
9.  'cnt_words_unique' - количество уникальных слов
10. 'unwords_words' - отношение количества уникальных слов к количеству слов
11. 'median_word_length' - медианная длина слов
12. 'mean_word_length' - средняя длина слова
13. 'max_word_length' - максимальная длина слова
14. 'words_symbols' - отношение количества слов к количеству символов
15. 'words_dots' - отношение количества слов к количеству точек
16. 'words_commas' - отношение количества слов к количеству запятых
17. 'words_excls' - отношение количества слов к количеству восклицательных знаков
18. 'words_questions' - отношение количества слов к количеству вопросительных знаков
19. 'words_semicolons' - отношение количества слов к количеству точек с запятой
20. 'words_colons' - отношение количества слов к количеству двоеточий
21. 'words_dashs' - отношение количества слов к количеству тире
22. 'words_aposts' - отношение количества слов к количеству апострофов
23. 'words_ellipsis' - отношение количества слов к количеству многоточий
24. 'words_quots' - отношение количества слов к количеству кавычек
25. 'cnt_adv_freq' - словарь с количествами частотных наречий
26. 'cnt_swadesh_freq' - словарь с количествами слов из списка Сводеша
27. 'cnt_word_eng' - количество уникальных слов из словаря англ. языка
28. 'prc_wrds_not_eng' - отношение количества английских слов к количеству слов
29. 'uniq_word_cnt' - словарь с количествами уникальных слов
30. 'cnt_punct_frq' - словарь с количествами знаков пунктуации
31. 'lex_div' - (lexical_diversity) - лексическое разнообразие
32. 'tfidf_keywords' - ключевые слова с максимальными TF-IDF значениями
33. 'pos_frq' - словарь с количествами слов по частям речи
34. 'pos_cnt' - количество уникальных частей речи
35. 'ent_frq' - словарь с количествами слов именнованных сущностей
36. 'ent_cnt' - количество уникальных именнованных сущностей
37. 'uchars_frq' - словарь с количествами букв английского алфавита
38. 'uchars_cnt' - количество уникальных букв английского алфавита
39. 'fk_score' - показатель уровня читаемости текста по формуле Flesch-Kincaid

Посмотрим на статистики числовых данных

In [ ]:
df_train.describe()

,cnt_sent,text_len,text_len2,words_cnt,wrds_sent_cnt,cnt_words_unique,unwords_words,median_word_length,mean_word_length,max_word_length,...,words_aposts,words_ellipsis,words_quots,cnt_word_eng,prc_wrds_not_eng,lex_div,pos_cnt,ent_cnt,uchars_cnt,fk_score
count,100.000000,1.000000e+02,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,6708.100000,5.931244e+05,5.717292e+05,106611.370000,17.625164,10367.320000,0.137389,4.0,4.343600,28.380000,...,inf,inf,inf,5530.500000,0.564276,0.136749,33.030000,16.160000,51.380000,7.331000
std,5023.105853,4.007281e+05,3.867990e+05,71497.536884,8.258601,5874.831163,0.086244,0.0,0.166706,19.324857,...,NaN,NaN,NaN,2678.349641,0.074756,0.086098,1.158674,1.587324,1.012847,3.428304
min,142.000000,1.089200e+04,1.055000e+04,2036.000000,6.748858,761.000000,0.053816,4.0,3.990000,13.000000,...,14.140523,61.212629,12.508128,526.000000,0.347497,0.053738,27.000000,11.000000,46.000000,1.800000
25%,2919.250000,2.144975e+05,2.062805e+05,38724.250000,12.528176,5885.250000,0.083371,4.0,4.250000,20.000000,...,43.932483,2388.984091,41.958527,3544.250000,0.504767,0.084104,33.000000,15.000000,51.000000,5.250000
50%,5438.500000,5.450980e+05,5.294670e+05,100340.500000,15.098669,9967.500000,0.110327,4.0,4.340000,23.000000,...,78.081368,NaN,591.032145,5706.500000,0.569669,0.108338,33.000000,17.000000,52.000000,6.650000
75%,9811.250000,9.919358e+05,9.493388e+05,184172.250000,20.497069,14322.000000,0.145839,4.0,4.450000,30.250000,...,NaN,NaN,NaN,7500.000000,0.618218,0.145886,34.000000,17.000000,52.000000,8.300000
max,19644.000000,1.227145e+06,1.184651e+06,199998.000000,56.632507,28415.000000,0.505560,4.0,4.930000,187.000000,...,inf,inf,inf,13685.000000,0.699397,0.509004,34.000000,18.000000,52.000000,22.100000


Посмотрим типы данных

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              100 non-null    object 
 1   text_               100 non-null    object 
 2   cnt_sent            100 non-null    int64  
 3   text_len            100 non-null    int64  
 4   text                100 non-null    object 
 5   text_len2           100 non-null    int64  
 6   words_cnt           100 non-null    int64  
 7   wrds_sent_cnt       100 non-null    float64
 8   cnt_words_unique    100 non-null    int64  
 9   unwords_words       100 non-null    float64
 10  median_word_length  100 non-null    float64
 11  mean_word_length    100 non-null    float64
 12  max_word_length     100 non-null    int64  
 13  words_symbols       100 non-null    float64
 14  words_dots          100 non-null    float64
 15  words_commas        100 non-null    float64
 16  words_exc

Выполним замену типов данных float64 в flaot16 для экономии памяти и времени обучения моделей

In [ ]:
def convert_float64_to_16(df):
    for column in df.columns:
        if df[column].dtype == 'float64':
            df[column] = df[column].astype('float16')
    return df


df_train = convert_float64_to_16(df_train)
df_test = convert_float64_to_16(df_test)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              51 non-null     object 
 1   text_               51 non-null     object 
 2   cnt_sent            51 non-null     int64  
 3   text_len            51 non-null     int64  
 4   text                51 non-null     object 
 5   text_len2           51 non-null     int64  
 6   words_cnt           51 non-null     int64  
 7   wrds_sent_cnt       51 non-null     float16
 8   cnt_words_unique    51 non-null     int64  
 9   unwords_words       51 non-null     float16
 10  median_word_length  51 non-null     float16
 11  mean_word_length    51 non-null     float16
 12  max_word_length     51 non-null     int64  
 13  words_symbols       51 non-null     float16
 14  words_dots          51 non-null     float16
 15  words_commas        51 non-null     float16
 16  words_excl

Попробуем собрать один вектор для сравнения по косинусной близости и обучения моделей из всех числовых признаков и значений словарей

In [ ]:
df_train.columns

Index(['author', 'text_', 'cnt_sent', 'text_len', 'text', 'text_len2',
       'words_cnt', 'wrds_sent_cnt', 'cnt_words_unique', 'unwords_words',
       'median_word_length', 'mean_word_length', 'max_word_length',
       'words_symbols', 'words_dots', 'words_commas', 'words_excls',
       'words_questions', 'words_semicolons', 'words_colons', 'words_dashs',
       'words_aposts', 'words_ellipsis', 'words_quots', 'cnt_adv_freq',
       'cnt_swadesh_freq', 'cnt_word_eng', 'prc_wrds_not_eng', 'uniq_word_cnt',
       'cnt_punct_frq', 'lex_div', 'tfidf_keywords', 'pos_frq', 'pos_cnt',
       'ent_frq', 'ent_cnt', 'uchars_frq', 'uchars_cnt', 'fk_score'],
      dtype='object')

Оставим только признаки с числовыми данными

In [ ]:
Xtrain = df_train[['cnt_sent', 'text_len', 'text_len2', 'words_cnt',
                   'wrds_sent_cnt', 'cnt_words_unique', 'unwords_words',
       'median_word_length', 'mean_word_length', 'max_word_length',
       'words_symbols', 'words_dots', 'words_commas', 'words_excls',
       'words_questions', 'words_semicolons', 'words_colons', 'words_dashs',
       'words_aposts', 'words_ellipsis', 'words_quots', 'cnt_adv_freq',
       'cnt_swadesh_freq', 'cnt_word_eng', 'prc_wrds_not_eng', 'uniq_word_cnt',
       'cnt_punct_frq', 'lex_div', 'pos_frq', 'pos_cnt',
       'ent_frq', 'ent_cnt', 'uchars_frq', 'uchars_cnt', 'fk_score']]

Заменим признаки с частотными словарями с постоянными последовательными значений ключей, на векторы с частотными значениями их значений

In [ ]:
Xtrain['cnt_adv_freq_vec'] = Xtrain['cnt_adv_freq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtrain['cnt_swadesh_freq_vec'] = Xtrain['cnt_swadesh_freq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtrain['cnt_punct_frq_vec'] = Xtrain['cnt_punct_frq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtrain['pos_frq_vec'] = Xtrain['pos_frq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtrain['ent_frq_vec'] = Xtrain['ent_frq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtrain['uchars_frq_vec'] = Xtrain['uchars_frq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))

Удалим признаки со словарями

In [ ]:
Xtrain = Xtrain[['cnt_sent', 'text_len', 'text_len2', 'words_cnt',
                   'wrds_sent_cnt', 'cnt_words_unique', 'unwords_words',
       'median_word_length', 'mean_word_length', 'max_word_length',
       'words_symbols', 'words_dots', 'words_commas', 'words_excls',
       'words_questions', 'words_semicolons', 'words_colons', 'words_dashs',
       'words_aposts', 'words_ellipsis', 'words_quots',
       'cnt_word_eng', 'prc_wrds_not_eng', 'lex_div', 'pos_cnt', 'ent_cnt',
       'uchars_cnt', 'fk_score', 'cnt_adv_freq_vec', 'cnt_swadesh_freq_vec',
       'cnt_punct_frq_vec', 'pos_frq_vec', 'ent_frq_vec', 'uchars_frq_vec']]

Объеденим все числовые признаки в один общий вектор

In [ ]:
numerical_columns = Xtrain.select_dtypes(include=['int64', 'float16'])

Xtrain['vec'] = numerical_columns.apply(lambda row: np.array(row), axis=1)

Оставим только полученные признаки с векторами

In [ ]:
Xtrain = Xtrain[['vec', 'cnt_adv_freq_vec', 'cnt_swadesh_freq_vec',
  'cnt_punct_frq_vec', 'pos_frq_vec', 'ent_frq_vec', 'uchars_frq_vec']]

Объеденим все полученные вектора

In [ ]:
def combine_vectors(row):
    arrays = [
        row['vec'],
        row['cnt_adv_freq_vec'],
        row['cnt_swadesh_freq_vec'],
        row['cnt_punct_frq_vec'],
        row['pos_frq_vec'],
        row['ent_frq_vec'],
        row['uchars_frq_vec']
    ]
    # Определяем максимальный размер среди всех массивов
    uniform_size = max(array.shape[0] for array in arrays)
    # Удлиняем каждый массив до самого длинного, добавляя нули в конце
    arrays = [np.pad(array, (0, uniform_size - array.shape[0]), 'constant') for array in arrays]
    # Объединяем массивы
    return np.vstack(arrays)

In [ ]:
Xtrain['vector'] = Xtrain.apply(combine_vectors, axis=1)

Возьмём только полученный общий вектор

In [ ]:
X_train = Xtrain['vector']

Выпоолним, те же действия для тестового датасета

In [ ]:
Xtest = df_test[['cnt_sent', 'text_len', 'text_len2', 'words_cnt',
                   'wrds_sent_cnt', 'cnt_words_unique', 'unwords_words',
       'median_word_length', 'mean_word_length', 'max_word_length',
       'words_symbols', 'words_dots', 'words_commas', 'words_excls',
       'words_questions', 'words_semicolons', 'words_colons', 'words_dashs',
       'words_aposts', 'words_ellipsis', 'words_quots', 'cnt_adv_freq',
       'cnt_swadesh_freq', 'cnt_word_eng', 'prc_wrds_not_eng', 'uniq_word_cnt',
       'cnt_punct_frq', 'lex_div', 'pos_frq', 'pos_cnt',
       'ent_frq', 'ent_cnt', 'uchars_frq', 'uchars_cnt', 'fk_score']]

In [ ]:
Xtest['cnt_adv_freq_vec'] = Xtest['cnt_adv_freq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtest['cnt_swadesh_freq_vec'] = Xtest['cnt_swadesh_freq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtest['cnt_punct_frq_vec'] = Xtest['cnt_punct_frq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtest['pos_frq_vec'] = Xtest['pos_frq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtest['ent_frq_vec'] = Xtest['ent_frq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))
Xtest['uchars_frq_vec'] = Xtest['uchars_frq'].apply(lambda x: np.array([v if v is not None else 0 for v in x.values()]))

In [ ]:
Xtest = Xtest[['cnt_sent', 'text_len', 'text_len2', 'words_cnt',
                   'wrds_sent_cnt', 'cnt_words_unique', 'unwords_words',
       'median_word_length', 'mean_word_length', 'max_word_length',
       'words_symbols', 'words_dots', 'words_commas', 'words_excls',
       'words_questions', 'words_semicolons', 'words_colons', 'words_dashs',
       'words_aposts', 'words_ellipsis', 'words_quots',
       'cnt_word_eng', 'prc_wrds_not_eng', 'lex_div', 'pos_cnt', 'ent_cnt',
       'uchars_cnt', 'fk_score', 'cnt_adv_freq_vec', 'cnt_swadesh_freq_vec',
       'cnt_punct_frq_vec', 'pos_frq_vec', 'ent_frq_vec', 'uchars_frq_vec']]

In [ ]:
numerical_columns = Xtest.select_dtypes(include=['int64', 'float16'])

Xtest['vec'] = numerical_columns.apply(lambda row: np.array(row), axis=1)
Xtest = Xtest[['vec', 'cnt_adv_freq_vec', 'cnt_swadesh_freq_vec',
  'cnt_punct_frq_vec', 'pos_frq_vec', 'ent_frq_vec', 'uchars_frq_vec']]
Xtest['vector'] = Xtest.apply(combine_vectors, axis=1)
X_test = Xtest['vector']

Заменим бесконечности на нули и перобразуем вектор в одноуровневый

In [ ]:
def inf_to_zerro(data):
    for arr in data:
        arr[np.isinf(arr)] = 0
    return data


X_train = X_train.apply(lambda x: inf_to_zerro(x))
X_test = X_test.apply(lambda x: inf_to_zerro(x))
X_train = X_train.apply(lambda x: np.concatenate(x))
X_test = X_test.apply(lambda x: np.concatenate(x))

Выберем целевые метки классов для обучения и проверки моделей

In [ ]:
ytrain = df_train['author']
ytest = df_test['author']

Напишем функцию для вывода результата обучения модели

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def calculate_multiclass_metrics(y_true, y_pred):
    """
    Рассчитывает основные метрики для многоклассовой классификации.

    :param y_true: Список или массив истинных значений классов.
    :param y_pred: Список или массив предсказанных значений классов.
    :return: Словарь с основными метриками.
    """

    metrics = {
        'accuracy': round(accuracy_score(y_true, y_pred), 4),
        'precision_macro': round(precision_score(y_true, y_pred, average='macro'), 4),
        'recall_macro': round(recall_score(y_true, y_pred, average='macro'), 4),
        'f1_macro': round(f1_score(y_true, y_pred, average='macro'), 4),
        'precision_micro': round(precision_score(y_true, y_pred, average='micro'), 4),
        'recall_micro': round(recall_score(y_true, y_pred, average='micro'), 4),
        'f1_micro': round(f1_score(y_true, y_pred, average='micro'), 4),
    }

    # Также можно вывести подробный отчет по каждому классу
    report = classification_report(y_true, y_pred)
    print("Classification Report:\n", report)

    return metrics

Попробуем получить предсказания классов на основе косинусной близости векторов объектов

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Преобразуем серии в numpy массивы
X_train_np = np.array(X_train.tolist())
X_test_np = np.array(X_test.tolist())

# Рассчитываем косинусную близость
similarity_matrix = cosine_similarity(X_test_np, X_train_np)

# Инициализируем список для предсказанных меток
y_pred = []

# Для каждого вектора в X_test находим наиболее близкий вектор в X_train
for i in range(len(X_test_np)):
    # Находим индекс наиболее близкого вектора в X_train
    closest_index = np.argmax(similarity_matrix[i])

    # Сохраняем соответствующую метку класса из ytrain
    predicted_label = ytrain.iloc[closest_index]

    # Добавляем метку в список предсказаний
    y_pred.append(predicted_label)

# Вызываем вашу функцию для оценки результатов
calculate_multiclass_metrics(ytest.tolist(), y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
        Ahmet_Hamdi_Tanpinar       0.00      0.00      0.00         0
               Aldous_Huxley       0.00      0.00      0.00         0
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       0.00      0.00      0.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
              Arthur_Rimbaud       0.00      0.00      0.00         0
                Beaumarchais       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00      

{'accuracy': 0.2745,
 'precision_macro': 0.2063,
 'recall_macro': 0.2222,
 'f1_macro': 0.2116,
 'precision_micro': 0.2745,
 'recall_micro': 0.2745,
 'f1_micro': 0.2745}

Попробуем обучить логистическую регрессию с StandardScaler

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Создаем конвейер для масштабирования данных и обучения модели
model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

# Обучаем модель на тренировочных данных
model.fit(X_train_np, ytrain)

# Делаем предсказания на тестовых данных
y_pred = model.predict(X_test_np)

# Оцениваем результаты
calculate_multiclass_metrics(ytest.tolist(), y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
               Aldous_Huxley       0.00      0.00      0.00         0
             Alexandre_Dumas       0.00      0.00      0.00         1
             Alphonse_Daudet       0.00      0.00      0.00         1
               Anton_Chekhov       1.00      1.00      1.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
                      Borges       0.00      0.00      0.00         0
             Boris_Pasternak       0.00      0.00      0.00         0
          Charles_Baudelaire       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
            Chingiz_Aitmatov       0.00      0.00      0.00      

{'accuracy': 0.2353,
 'precision_macro': 0.1627,
 'recall_macro': 0.1875,
 'f1_macro': 0.1663,
 'precision_micro': 0.2353,
 'recall_micro': 0.2353,
 'f1_micro': 0.2353}

Попробуем обучить логистическую регрессию

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Создаем конвейер для масштабирования данных и обучения модели
model = LogisticRegression(max_iter=1000)

# Обучаем модель на тренировочных данных
model.fit(X_train_np, ytrain)

# Делаем предсказания на тестовых данных
y_pred = model.predict(X_test_np)

# Оцениваем результаты
calculate_multiclass_metrics(ytest.tolist(), y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
        Ahmet_Hamdi_Tanpinar       0.00      0.00      0.00         0
               Aldous_Huxley       0.00      0.00      0.00         0
             Alexandre_Dumas       0.00      0.00      0.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       0.00      0.00      0.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
              Bertolt_Brecht       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       0.00      0.00      0.00      

{'accuracy': 0.2549,
 'precision_macro': 0.1339,
 'recall_macro': 0.2131,
 'f1_macro': 0.155,
 'precision_micro': 0.2549,
 'recall_micro': 0.2549,
 'f1_micro': 0.2549}

Попробуем уменьшить размерности и найдём косинусную близость

In [ ]:
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

# Преобразуем серии в numpy массивы
X_train_np = np.array(X_train.tolist())
X_test_np = np.array(X_test.tolist())

# Применяем PCA для уменьшения размерности
pca = PCA(n_components=35)  # Оставляем достаточное количество компонент для объяснения 95% дисперсии
X_train_pca = pca.fit_transform(X_train_np)
X_test_pca = pca.transform(X_test_np)

# Рассчитываем косинусную близость на уменьшенной размерности
similarity_matrix = cosine_similarity(X_test_pca, X_train_pca)

# Инициализируем список для предсказанных меток
y_pred = []

# Для каждого вектора в X_test находим наиболее близкий вектор в X_train
for i in range(len(X_test_pca)):
    # Находим индекс наиболее близкого вектора в X_train
    closest_index = np.argmax(similarity_matrix[i])

    # Сохраняем соответствующую метку класса из ytrain
    predicted_label = ytrain.iloc[closest_index]

    # Добавляем метку в список предсказаний
    y_pred.append(predicted_label)

# Вызываем вашу функцию для оценки результатов
calculate_multiclass_metrics(ytest.tolist(), y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
        Ahmet_Hamdi_Tanpinar       0.00      0.00      0.00         0
               Aldous_Huxley       0.00      0.00      0.00         0
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       0.00      0.00      0.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
              Arthur_Rimbaud       0.00      0.00      0.00         0
                Beaumarchais       0.00      0.00      0.00         0
             Boris_Pasternak       0.00      0.00      0.00         0
          Charles_Baudelaire       0.00      0.00      0.00      

{'accuracy': 0.3137,
 'precision_macro': 0.2424,
 'recall_macro': 0.2424,
 'f1_macro': 0.2424,
 'precision_micro': 0.3137,
 'recall_micro': 0.3137,
 'f1_micro': 0.3137}

Попробуем обучить логистическую регрессию с настроенным гиперпараметром C

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Создаем конвейер для масштабирования данных и обучения модели
model = LogisticRegression(max_iter=1000, C=1.015)

# Обучаем модель на тренировочных данных
model.fit(X_train_pca, ytrain)

# Делаем предсказания на тестовых данных
y_pred = model.predict(X_test_pca)

# Оцениваем результаты
calculate_multiclass_metrics(ytest.tolist(), y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
        Ahmet_Hamdi_Tanpinar       0.00      0.00      0.00         0
               Aldous_Huxley       0.00      0.00      0.00         0
             Alexandre_Dumas       0.00      0.00      0.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       0.00      0.00      0.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
              Arthur_Rimbaud       0.00      0.00      0.00         0
                Beaumarchais       0.00      0.00      0.00         0
             Boris_Pasternak       0.00      0.00      0.00         0
          Charles_Baudelaire       0.00      0.00      0.00      

{'accuracy': 0.1765,
 'precision_macro': 0.1061,
 'recall_macro': 0.1364,
 'f1_macro': 0.1162,
 'precision_micro': 0.1765,
 'recall_micro': 0.1765,
 'f1_micro': 0.1765}

Обучим LinearSVC с StandardScaler

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

model = make_pipeline(
                      StandardScaler(),
                      LinearSVC(max_iter=1000, C=0.99))

model.fit(X_train_np, ytrain)
y_pred = model.predict(X_test_np)

calculate_multiclass_metrics(ytest, y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
             Alexandre_Dumas       0.00      0.00      0.00         1
             Alphonse_Daudet       0.00      0.00      0.00         1
               Anton_Chekhov       1.00      1.00      1.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
                      Borges       0.00      0.00      0.00         0
             Boris_Pasternak       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
            Chingiz_Aitmatov       0.00      0.00      0.00         0
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       0.00      0.00      0.00      

{'accuracy': 0.2353,
 'precision_macro': 0.1877,
 'recall_macro': 0.2034,
 'f1_macro': 0.1889,
 'precision_micro': 0.2353,
 'recall_micro': 0.2353,
 'f1_micro': 0.2353}

Обучим RidgeClassifier с StandardScaler

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

model = make_pipeline(StandardScaler(),
                      RidgeClassifier(max_iter=1000))

model.fit(X_train_np, ytrain)
y_pred = model.predict(X_test_np)

calculate_multiclass_metrics(ytest, y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
             Alexandre_Dumas       0.00      0.00      0.00         1
             Alphonse_Daudet       0.00      0.00      0.00         1
               Anton_Chekhov       1.00      1.00      1.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
              Bertolt_Brecht       0.00      0.00      0.00         0
                      Borges       0.00      0.00      0.00         0
             Boris_Pasternak       0.00      0.00      0.00         0
          Charles_Baudelaire       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
            Chingiz_Aitmatov       0.00      0.00      0.00      

{'accuracy': 0.2353,
 'precision_macro': 0.1815,
 'recall_macro': 0.1935,
 'f1_macro': 0.1839,
 'precision_micro': 0.2353,
 'recall_micro': 0.2353,
 'f1_micro': 0.2353}

Обучим MultinomialNB с MinMaxScaler

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

model = make_pipeline(MinMaxScaler(),
                      # StandardScaler(),
                      MultinomialNB())

model.fit(X_train_pca, ytrain)

y_pred = model.predict(X_test_pca)

calculate_multiclass_metrics(ytest, y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
        Ahmet_Hamdi_Tanpinar       0.00      0.00      0.00         0
                Albert_Camus       0.00      0.00      0.00         0
               Aldous_Huxley       0.00      0.00      0.00         0
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       0.00      0.00      0.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
              Bertolt_Brecht       0.00      0.00      0.00         0
             Boris_Pasternak       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00      

{'accuracy': 0.3529,
 'precision_macro': 0.25,
 'recall_macro': 0.2687,
 'f1_macro': 0.2547,
 'precision_micro': 0.3529,
 'recall_micro': 0.3529,
 'f1_micro': 0.3529}

Обучим MultinomialNB с TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import joblib

vectorizer = TfidfVectorizer()
model = MultinomialNB()

pipeline = make_pipeline(vectorizer, model)

pipeline.fit(df_train['text'], ytrain)
y_pred = pipeline.predict(df_test['text'])

calculate_multiclass_metrics(ytest.tolist(), y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
             Alexandre_Dumas       0.50      1.00      0.67         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       0.00      0.00      0.00         1
                Aristophanes       1.00      1.00      1.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
                      Borges       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       0.50      1.00      0.67         1
          F_Scott_Fitzgerald       0.00      0.00      0.00         1
                 Franz_Kafka       1.00      1.00      1.00      

{'accuracy': 0.4902,
 'precision_macro': 0.383,
 'recall_macro': 0.4464,
 'f1_macro': 0.4,
 'precision_micro': 0.4902,
 'recall_micro': 0.4902,
 'f1_micro': 0.4902}

Попробуем обучить GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

model = make_pipeline(#StandardScaler(),
                      GaussianNB())

model.fit(X_train_pca, ytrain)

y_pred = model.predict(X_test_pca)

calculate_multiclass_metrics(ytest, y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
             Alexandre_Dumas       0.00      0.00      0.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
    Antoine_De_Saint_Exupery       0.00      0.00      0.00         0
               Anton_Chekhov       0.50      1.00      0.67         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
              Arthur_Rimbaud       0.00      0.00      0.00         0
             Boris_Pasternak       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       0.00      0.00      0.00      

{'accuracy': 0.3137,
 'precision_macro': 0.2077,
 'recall_macro': 0.2462,
 'f1_macro': 0.2174,
 'precision_micro': 0.3137,
 'recall_micro': 0.3137,
 'f1_micro': 0.3137}

Обучим LogisticRegression с MinMaxScaler на данных после PCA

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Создаем конвейер для масштабирования данных и обучения модели
model = make_pipeline(MinMaxScaler(),
                      LogisticRegression(max_iter=1000))
# Обучаем модель на тренировочных данных
model.fit(X_train_pca, ytrain)

# Делаем предсказания на тестовых данных
y_pred = model.predict(X_test_pca)

# Оцениваем результаты
calculate_multiclass_metrics(ytest.tolist(), y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
        Ahmet_Hamdi_Tanpinar       0.00      0.00      0.00         0
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       0.00      0.00      0.00         1
               Anton_Chekhov       0.00      0.00      0.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
             Boris_Pasternak       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       0.00      0.00      0.00         1
          F_Scott_Fitzgerald       0.33      1.00      0.50      

{'accuracy': 0.3137,
 'precision_macro': 0.2556,
 'recall_macro': 0.2667,
 'f1_macro': 0.2583,
 'precision_micro': 0.3137,
 'recall_micro': 0.3137,
 'f1_micro': 0.3137}

Добавим векторизацию текстов, при помощи мешка слов

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vec = CountVectorizer(lowercase=False, token_pattern=r'(?u)\b\w+\b')
X = cnt_vec.fit_transform(df_train.text)

In [ ]:
res = dict(sorted(cnt_vec.vocabulary_.items(), key=lambda item: item[1]))
pd.DataFrame(X.toarray(), columns=res.keys())

,A,AAA,AAAgnes,AACR,AAs,AB,ABAFT,ABANDON,ABANDONED,ABBA,...,Şerefinize,Şişhane,Şânizâde,Şîr,Şükrü,şalvars,şarkı,şerike,сovering,сzerov
0,68,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,105,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,413,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,103,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,198,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,0,0
98,145,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Обучим LogisticRegression на BoW

In [ ]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler

vec = CountVectorizer(ngram_range=(1, 1))

bow = vec.fit_transform(df_train.text)
bow_test = vec.transform(df_test.text)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(bow, ytrain)
y_pred = clf.predict(bow_test)

calculate_multiclass_metrics(ytest.tolist(), y_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
    Antoine_De_Saint_Exupery       0.00      0.00      0.00         0
               Anton_Chekhov       0.00      0.00      0.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
             Boris_Pasternak       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       0.00      0.00      0.00         1
          F_Scott_Fitzgerald       0.00      0.00      0.00      

{'accuracy': 0.4118,
 'precision_macro': 0.3684,
 'recall_macro': 0.3684,
 'f1_macro': 0.3684,
 'precision_micro': 0.4118,
 'recall_micro': 0.4118,
 'f1_micro': 0.4118}

Создадим датафреймы с BoW

In [ ]:
bow_train_df = pd.DataFrame(bow.toarray(), columns=vec.get_feature_names_out())
bow_test_df = pd.DataFrame(bow_test.toarray(), columns=vec.get_feature_names_out())

Обучим LogisticRegression на векторах полученных со статистик и обработанных PCA объединённых с BoW

In [ ]:
%%time
X_train_pca_df = pd.DataFrame(X_train_pca)
X_test_pca_df = pd.DataFrame(X_test_pca)
bow_train_df = pd.concat([bow_train_df, X_train_pca_df], axis=1)
bow_test_df = pd.concat([bow_test_df, X_test_pca_df], axis=1)
bow_train_df.columns = bow_train_df.columns.astype(str)
bow_test_df.columns = bow_test_df.columns.astype(str)

scaler = MaxAbsScaler()
bow_train_df = scaler.fit_transform(bow_train_df)
bow_test_df = scaler.transform(bow_test_df)

clf = LogisticRegression(max_iter=333, random_state=42)
clf.fit(bow_train_df, ytrain)
bow_vec_pred = clf.predict(bow_test_df)

calculate_multiclass_metrics(ytest.tolist(), bow_vec_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.00      0.00      0.00         1
             Agatha_Christie       0.00      0.00      0.00         1
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
    Antoine_De_Saint_Exupery       0.00      0.00      0.00         0
               Anton_Chekhov       0.00      0.00      0.00         1
                Aristophanes       0.00      0.00      0.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
             Boris_Pasternak       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       0.00      0.00      0.00         1
          F_Scott_Fitzgerald       0.00      0.00      0.00      

{'accuracy': 0.4118,
 'precision_macro': 0.3684,
 'recall_macro': 0.3684,
 'f1_macro': 0.3684,
 'precision_micro': 0.4118,
 'recall_micro': 0.4118,
 'f1_micro': 0.4118}

Выполним Tfidf векторизацию текстов

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(lowercase=False, token_pattern=r'(?u)\b\w+\b')
X2 = tfidf_vec.fit_transform(df_train.text)

In [ ]:
res = dict(sorted(cnt_vec.vocabulary_.items(), key=lambda item: item[1]))
pd.DataFrame(X2.toarray(), columns=res.keys())

Оставим для эксперементов только тексты

In [ ]:
train = df_train.text
test = df_test.text

In [ ]:
del df_train, df_test

Попробуем обучить LogisticRegression на Tfidf с биграммами, обучить не вышло, из-за нехватки памяти

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression

vec = TfidfVectorizer(ngram_range=(2, 2))
vec_train = vec.fit_transform(train)
vec_test = vec.transform(test)

scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

clf = LogisticRegression(max_iter=333, random_state=42)
clf.fit(vec_train, ytrain)
pred_tfidf = clf.predict(vec_test)

calculate_multiclass_metrics(ytest.tolist(), pred_tfidf)

NameError: name 'LogisticRegression' is not defined

Лучшая модель TfidfVectorizer(ngram_range=(1, 2)) с OneVsRestClassifier, MaxAbsScaler и LR

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
import joblib

vectorizer = TfidfVectorizer(ngram_range=(1, 2))
base_model = LogisticRegression(solver='liblinear')
ovr = OneVsRestClassifier(base_model)
scaler = MaxAbsScaler()
pipeline = make_pipeline(vectorizer, scaler, ovr)

pipeline.fit(train, ytrain)
lr_pred = pipeline.predict(test)

joblib.dump(pipeline, 'pipeline.pkl')

lr_pred_prob = pipeline.predict_proba(test)
lr_probs = lr_pred_prob[:, 1]

calculate_multiclass_metrics(ytest.tolist(), lr_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.50      1.00      0.67         1
             Agatha_Christie       1.00      1.00      1.00         1
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       1.00      1.00      1.00         1
                Aristophanes       1.00      1.00      1.00         1
          Arthur_Conan_Doyle       1.00      1.00      1.00         1
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       1.00      1.00      1.00         1
            Ernest_Hemingway       0.00      0.00      0.00         0
          F_Scott_Fitzgerald       1.00      1.00      1.00         1
                 Franz_Kafka       1.00      1.00      1.00      

{'accuracy': 0.7255,
 'precision_macro': 0.6152,
 'recall_macro': 0.6727,
 'f1_macro': 0.6333,
 'precision_micro': 0.7255,
 'recall_micro': 0.7255,
 'f1_micro': 0.7255}

Попробуем заменить LR в пайплайне прошлой модели на MultinomialNB

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline

vectorizer = TfidfVectorizer(ngram_range=(1, 1))
base_model = MultinomialNB()
ovr = OneVsRestClassifier(base_model)
scaler = MaxAbsScaler()
pipeline = make_pipeline(vectorizer, ovr)

pipeline.fit(df_train['text'], ytrain)
nb_pred = pipeline.predict(df_test['text'])

nb_pred_prob = pipeline.predict_proba(df_test['text'])

calculate_multiclass_metrics(ytest.tolist(), nb_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.50      1.00      0.67         1
             Agatha_Christie       0.00      0.00      0.00         1
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       1.00      1.00      1.00         1
                Aristophanes       1.00      1.00      1.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
                      Borges       0.00      0.00      0.00         0
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       0.00      0.00      0.00         1
          F_Scott_Fitzgerald       0.00      0.00      0.00         1
                 Franz_Kafka       1.00      1.00      1.00      

{'accuracy': 0.4902,
 'precision_macro': 0.3929,
 'recall_macro': 0.4464,
 'f1_macro': 0.4071,
 'precision_micro': 0.4902,
 'recall_micro': 0.4902,
 'f1_micro': 0.4902}

Попробуем стекинг лучших моделей, в стандартном варианте не проходило, так как у нас по 1 объекту каждого класса

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split

nb_pipeline = make_pipeline(
    TfidfVectorizer(),
    MultinomialNB()
)
nb_pipeline.fit(df_train['text'], ytrain)

lr_pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 1)),
    MaxAbsScaler(),
    OneVsRestClassifier(LogisticRegression(solver='liblinear'))
)
lr_pipeline.fit(df_train['text'], ytrain)

bow_pipeline = make_pipeline(
    CountVectorizer(ngram_range=(1, 1)),
    MaxAbsScaler(),
    OneVsRestClassifier(LogisticRegression(solver='liblinear'))
)
bow_pipeline.fit(df_train['text'], ytrain)

stacking_clf = StackingClassifier(
    estimators=[
        ('nb', nb_pipeline),
        ('lr', lr_pipeline),
        #('bow', bow_pipeline)
    ],
    final_estimator=OneVsRestClassifier(LogisticRegression(solver='liblinear')),#OneVsRestClassifier(LogisticRegression(solver='liblinear')
    cv='prefit'  # Использование без кросс-валидации, так как модели уже предобучены
)

stacking_clf.fit(df_train['text'], ytrain)
st_pred = stacking_clf.predict(df_test['text'])

calculate_multiclass_metrics(ytest.tolist(), st_pred)

Посмотрим вырастит ли качество, если сложить вероятности классов лучших моделей и поделить их пополам. Качество не менялось

In [ ]:
average_probs = (nb_pred_prob + lr_pred_prob) / 2
pred_from_prob = np.argmax(average_probs, axis=1)
class_labels = pipeline.named_steps['onevsrestclassifier'].classes_
average_pred_classes = [class_labels[index] for index in pred_from_prob]

calculate_multiclass_metrics(ytest.tolist(), average_pred_classes)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       1.00      1.00      1.00         1
             Agatha_Christie       0.00      0.00      0.00         1
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       1.00      1.00      1.00         1
                Aristophanes       1.00      1.00      1.00         1
          Arthur_Conan_Doyle       0.00      0.00      0.00         1
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       1.00      1.00      1.00         1
            Ernest_Hemingway       0.00      0.00      0.00         0
          F_Scott_Fitzgerald       0.50      1.00      0.67         1
                 Franz_Kafka       1.00      1.00      1.00      

{'accuracy': 0.5882,
 'precision_macro': 0.4322,
 'recall_macro': 0.5085,
 'f1_macro': 0.4576,
 'precision_micro': 0.5882,
 'recall_micro': 0.5882,
 'f1_micro': 0.5882}

Напишем код для инференса лучшей модели

In [ ]:
import joblib

pipeline = joblib.load('pipeline.pkl')

def predict_top_3_authors(text):
    text_data = [text]
    predicted_probs = pipeline.predict_proba(text_data)
    top_3_indices = np.argsort(predicted_probs[0])[-3:][::-1]
    top_3_probs = predicted_probs[0][top_3_indices]
    top_3_authors = pipeline.named_steps['onevsrestclassifier'].classes_[top_3_indices]
    for author, prob in zip(top_3_authors, top_3_probs):
        print(f"Author: {author}, Probability: {prob}")

example_text ='''
One ring to rule them all, one ring to find them,
One ring to bring them all and in the darkness bind them.'''
predict_top_3_authors(example_text)

Author: Antoine_De_Saint_Exupery, Probability: 0.01396337039015265
Author: J_R_R_Tolkien, Probability: 0.013169534324720733
Author: Aesop, Probability: 0.012547013893284076


Далее идут ячейки с экспериментами выполненными в ноутбуке на kaggle с большей ОЗУ и с использованием GPU, там мной были полученны вектора W2V, TinyBert, SentanceBert, но модели LR и Catboost на этих признаках по отдельности, вместе и с различными комбинациями с включениями TFidf и вектора на статистиках не показали более высокого результата. Возможно, что результат удастся улучшить, если попробовать дообучить лучшую модель на каких-то векторах из этих экспериментов.

Попробуем заменить вектора из словарей на их статистики

In [ ]:
def process_vector_columns(df):
    new_columns = {}

    for col in df.columns:
        # Проверка наличия списка/вектора в каждой ячейке
        if df[col].apply(lambda x: isinstance(x, object)).all():
            # Вычисление медианных, средних и максимальных значений
            new_columns[f'{col}_median'] = df[col].apply(np.median)
            new_columns[f'{col}_mean'] = df[col].apply(np.mean)
            new_columns[f'{col}_max'] = df[col].apply(np.max)

    # Удаляем оригинальные столбцы с векторами
    df.drop(columns=[col for col in df.columns if col in new_columns.keys()], inplace=True)

    # Добавляем новые столбцы в DataFrame
    for new_col, values in new_columns.items():
        df[new_col] = values

    return df


X_train = process_vector_columns(X_train)
X_train = X_train.fillna(0)
X_test = process_vector_columns(X_test)
X_test = X_test.fillna(0)

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    random_seed=33,
    depth=6,
    loss_function='MultiClass',
    task_type='GPU',
    devices='0:1',
    verbose=10
)

model.fit(X_train, ytrain)
cb_pred = model.predict(X_test)

calculate_multiclass_metrics(ytest.tolist(), cb_pred)

Получил эмбеддинги при помощи SentenceTransformer

In [ ]:
pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

train_embeddings = model.encode(df_train['text'].tolist())
df_train['embeddings'] = list(train_embeddings)

test_embeddings = model.encode(df_test['text'].tolist())
df_test['embeddings'] = list(test_embeddings)

Пробовал отобрать наиболее значимые признаки, но не вышло, повисла ячейка

In [ ]:
from sklearn.model_selection import KFold

# Используйте обычную KFold-кросс-валидацию вместо StratifiedKFold
cv = KFold(n_splits=3)

# Сохранение остальных шагов без изменений
model = LogisticRegression(max_iter=1000)
rfecv = RFECV(estimator=model, step=1, cv=cv, scoring='f1_weighted', min_features_to_select=1)

# Подгонка RFECV
rfecv.fit(train_lr, ytrain)

# Получение отобранных признаков
selected_features = rfecv.support_
X_selected = train_lr.loc[:, selected_features]

# Вывести выбранные признаки и их количество
print("Выбранные признаки:", X_selected.columns.tolist())
print("Оптимальное количество признаков:", rfecv.n_features_)

Получил эмбеддинги при помощи TinyBERT

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

model_name = "huawei-noah/TinyBERT_General_6L_768D"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def encode_texts(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

train_embs = encode_texts(df_train['text'].tolist())
df_train['tiny_embs'] = list(train_embs)

test_embs = encode_texts(df_test['text'].tolist())
df_test['tiny_embs'] = list(test_embs)

Получил эмбеддинги при помощи Word2Vec

In [ ]:
from gensim.models import Word2Vec
from sklearn.preprocessing import normalize

def preprocess_texts(texts):
    return [text.lower().split() for text in texts]

texts = preprocess_texts(df_train['text'])

# Обучаем модель Word2Vec
w2v_model = Word2Vec(sentences=texts, vector_size=300, window=5, min_count=1, workers=4)

# Функция для получения векторного представления текста
def text_to_vector(text, model):
    words = text.lower().split()
    vector = np.mean([model.wv[word] for word in words if word in model.wv], axis=0)
    return vector if vector is not np.nan else np.zeros(model.vector_size)

df_train['w2v_embs'] = df_train['text'].apply(lambda x: text_to_vector(x, w2v_model))
df_train['w2v_embs'] = df_train['w2v_embs'].apply(lambda x: normalize([x])[0])

df_test['w2v_embs'] = df_test['text'].apply(lambda x: text_to_vector(x, w2v_model))
df_test['w2v_embs'] = df_test['w2v_embs'].apply(lambda x: normalize([x])[0])

Собрал в отдельные датасеты все полученные векторы

In [ ]:
vec_train_df['sent_embs'] = df_train['embeddings']
vec_test_df['sent_embs'] = df_test['embeddings']
vec_train_df['tiny_embs'] = df_train['tiny_embs']
vec_test_df['tiny_embs'] = df_test['tiny_embs']
vec_train_df['w2v_embs'] = df_train['w2v_embs']
vec_test_df['w2v_embs'] = df_test['w2v_embs']
vec_train_df['vecs'] = X_train
vec_test_df['vecs'] = X_test

Пробовал подовать модели векторы разбитые на отдельные столбцы

In [ ]:
new_columns = vec_train_df['sent_embs'].apply(pd.Series)
new_columns.columns = [f'sent_embs_{i}' for i in range(new_columns.shape[1])]
vec_train_df = pd.concat([vec_train_df, new_columns], axis=1)
vec_train_df.drop('sent_embs', axis=1, inplace=True)

new_columns = vec_train_df['tiny_embs'].apply(pd.Series)
new_columns.columns = [f'tiny_embs_{i}' for i in range(new_columns.shape[1])]
vec_train_df = pd.concat([vec_train_df, new_columns], axis=1)
vec_train_df.drop('tiny_embs', axis=1, inplace=True)

new_columns = vec_test_df['sent_embs'].apply(pd.Series)
new_columns.columns = [f'sent_embs_{i}' for i in range(new_columns.shape[1])]
vec_test_df = pd.concat([vec_test_df, new_columns], axis=1)
vec_test_df.drop('sent_embs', axis=1, inplace=True)

new_columns = vec_test_df['tiny_embs'].apply(pd.Series)
new_columns.columns = [f'tiny_embs_{i}' for i in range(new_columns.shape[1])]
vec_test_df = pd.concat([vec_test_df, new_columns], axis=1)
vec_test_df.drop('tiny_embs', axis=1, inplace=True)

Пробовал объединять все полученные векторы в один

In [ ]:
def clean_and_eval(s):
    # Преобразуем в строку, если данные еще не строки
    if isinstance(s, str):
        cleaned_str = s.replace('\x00', '')
        return eval(cleaned_str)
    # Предполагаем, что если не строка, то это уже массив
    return s

# Применение функции к каждому элементу
test['vecs'] = test['vecs'].apply(clean_and_eval)
test['sent_embs'] = test['sent_embs'].apply(clean_and_eval)
test['tiny_embs'] = test['tiny_embs'].apply(clean_and_eval)
test['w2v_embs'] = test['w2v_embs'].apply(clean_and_eval)

# Конкатенация эмбеддингов
test['combine'] = test.apply(lambda row: np.concatenate([
    np.array(row['vecs']),
    np.array(row['sent_embs']),
    np.array(row['tiny_embs']),
    np.array(row['w2v_embs'])
]), axis=1)